In [84]:
using GLPK
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using Random
using DelimitedFiles
using HTTP
using JSON
using LightXML
using SBML

## Loading Single-KnockOut Data

In [76]:
file_path = "../Data/Knock-Out Experiments - EcoCyc/LB enriched/LB_enriched_-_G.txt";
totaltime, totallines = open(file_path) do f
    global lines = readlines(f);
end

3059-element Vector{String}:
 "All-Genes\tNames\tObject ID\tProduct\tKnockout-Growth-Observations"
 "G7686\taaeA // yhcQ // b3241 // ECK3230\tG7686\tG7686-MONOMER\tOBS0-439 // OBS0-49 // OBS0-37 // OBS0-33 // OBS0-44"
 "G7685\taaeB // yhcP // b3240 // ECK3229\tG7685\tG7685-MONOMER\tOBS0-439 // OBS0-49 // OBS0-37 // OBS0-33 // OBS0-44"
 "G7688\taaeR // yhcS // qseA // b3243 // ECK3232\tG7688\tG7688-MONOMER\tOBS0-439 // OBS0-49 // OBS0-37 // OBS0-33 // OBS0-44"
 "G7687\taaeX // yhcR // b3242 // ECK3231\tG7687\tG7687-MONOMER\tOBS0-439 // OBS0-49 // OBS0-37 // OBS0-33 // OBS0-44"
 "EG11679\taas // b2836 // ECK2832\tEG11679\tAAS-MONOMER\tOBS0-439 // OBS0-49 // OBS0-40 // OBS0-37 // OBS0-33 // OBS0-44"
 "EG11112\taat // ycaA // b0885 // ECK0876\tEG11112\tEG11112-MONOMER\tOBS0-439 // OBS0-49 // OBS0-37 // OBS0-33 // OBS0-44"
 "G6670\tabgA // ydaJ // b1338 // ECK1334\tG6670\tG6670-MONOMER\tOBS0-439 // OBS0-49 // OBS0-37 // OBS0-33 // OBS0-44"
 "G6669\tabgB // ydaI // b1337 // ECK1333\tG6669\tG

In [53]:
function get_fields_name(fields_str)
    repr_str = repr(fields_str)[2:end-1];
    fields = split(repr_str, "\\t");
    return fields
end

get_fields_name (generic function with 1 method)

In [71]:
function get_line_dict(line_str, fields)
    repr_str = repr(line_str)[2:end-1];
    line_parts = split(repr_str, "\\t");
    line_dict = Dict(zip(fields, line_parts));
    return line_dict
end

get_line_dict (generic function with 1 method)

In [190]:
data_fields = get_fields_name(lines[1]);
num_data_lines = length(lines) - 1;
growth_genes_list = Vector{String}();
for i = 2:num_data_lines+1
    growth_gene_dict = get_line_dict(lines[i], data_fields);
    # growth_gene_id = growth_gene_dict["Object ID"];
    growth_gene_names = growth_gene_dict["Names"];
    push!(growth_genes_list, growth_gene_names);
end

In [191]:
growth_genes_list[1:10]

10-element Vector{String}:
 "aaeA // yhcQ // b3241 // ECK3230"
 "aaeB // yhcP // b3240 // ECK3229"
 "aaeR // yhcS // qseA // b3243 // ECK3232"
 "aaeX // yhcR // b3242 // ECK3231"
 "aas // b2836 // ECK2832"
 "aat // ycaA // b0885 // ECK0876"
 "abgA // ydaJ // b1338 // ECK1334"
 "abgB // ydaI // b1337 // ECK1333"
 "abgR // b1339 // ECK1335"
 "abgT // ydaH // b1336 // ECK1332"

In [196]:
function get_list_of_names(names_str)
    line_parts = split(names_str, " // ");
    return line_parts
end

get_list_of_names (generic function with 1 method)

In [197]:
get_list_of_names(growth_genes_list[1])

4-element Vector{SubString{String}}:
 "aaeA"
 "yhcQ"
 "b3241"
 "ECK3230"

In [200]:
growth_genes_standard_names = Vector{String}();
for growth_gene in growth_genes_list
    gene_standard_name = get_list_of_names(growth_gene);
    push!(growth_genes_standard_names, gene_standard_name[1]);
end

In [201]:
growth_genes_standard_names

3058-element Vector{String}:
 "aaeA"
 "aaeB"
 "aaeR"
 "aaeX"
 "aas"
 "aat"
 "abgA"
 "abgB"
 "abgR"
 "abgT"
 "abpA"
 "abpB"
 "abrB"
 ⋮
 "zinT"
 "zitB"
 "zntA"
 "zntB"
 "zntR"
 "znuA"
 "znuB"
 "znuC"
 "zraR"
 "zraS"
 "zupT"
 "zur"

### BiGG Model

In [ ]:
metabolites_filename = "../Data/BiGG Universal Model/bigg_models_metabolites.txt"; 
reactions_filename = "../Data/BiGG Universal Model/bigg_models_reactions.txt";
model_filename = "../Data/BiGG Universal Model/bigg_models_reactions.txt";

### Stoichiometric Data

In [89]:
sbml_filename = "../Data/Escherichia coli str. K-12 substr. MG1655/iAF1260b.xml"; 
mdl = readSBML(sbml_filename);

Dict{String, SBML.Compartment} with 3 entries:
  "c" => Compartment("cytosol", true, nothing, nothing, nothing, nothing, nothi…
  "e" => Compartment("extracellular space", true, nothing, nothing, nothing, no…
  "p" => Compartment("periplasm", true, nothing, nothing, nothing, nothing, not…

In [141]:
metabolites_list, reactions_list, S = SBML.stoichiometry_matrix(mdl);

In [140]:
Matrix(S)

1668×2388 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

In [142]:
reactions = mdl.reactions;  # reactants, products, lb, ub, oc, gene_product_association, kinetic_math, reversible, notes, annotation
metabolites = mdl.species;  # name, compartment, boundary_condition, formula, charge, initial_amount, initial_concentration, only_substance_units, notes, annotation
gene_products = mdl.gene_products;  # name, label, notes, annotation

In [143]:
reactions["R_AAMYL"]

SBML.Reaction(Dict("M_14glucan_c" => 1.0), Dict("M_malthx_c" => 1.0), (0.0, "[fbc]"), (999999.0, "[fbc]"), 0.0, SBML.GPARef("G_b1927"), nothing, false, nothing, "<sbml:annotation xmlns:sbml=\"http://www.sbml.org/sbml/level3/version1/core\">\n  <rdf:RDF xmlns:rdf=\"http://www.w3.org/1999/02/22-rdf-syntax-ns#\">\n    <rdf:Description rdf:about=\"#R_AAMYL\">\n      <bqbiol:is xmlns:bqbiol=\"http://biomodels.net/biology-qualifiers/\">\n        <rdf:Bag>\n          <rdf:li rdf:resource=\"http://identifiers.org/bigg.reaction/AAMYL\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/metanetx.reaction/MNXR95153\"/>\n        </rdf:Bag>\n      </bqbiol:is>\n    </rdf:Description>\n  </rdf:RDF>\n</sbml:annotation>")

In [144]:
gene_products["G_b1927"]

SBML.GeneProduct("amyA", "b1927", nothing, "<sbml:annotation xmlns:sbml=\"http://www.sbml.org/sbml/level3/version1/core\">\n  <rdf:RDF xmlns:rdf=\"http://www.w3.org/1999/02/22-rdf-syntax-ns#\">\n    <rdf:Description rdf:about=\"#G_b1927\">\n      <bqbiol:is xmlns:bqbiol=\"http://biomodels.net/biology-qualifiers/\">\n        <rdf:Bag>\n          <rdf:li rdf:resource=\"http://identifiers.org/asap/ABE-0006414\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/ecogene/EG11387\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/ncbigene/946434\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/ncbigi/16129874\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/refseq_locus_tag/b1927\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/refseq_name/amyA\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/refseq_synonym/yedC\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/refseq_synonym/ECK1926\"/>\n          <rdf:li rdf:resou

In [145]:
metabolites["M_14glucan_c"]

SBML.Species("1,4-alpha-D-glucan", "c", false, "C36H62O31", 0, nothing, nothing, false, nothing, "<sbml:annotation xmlns:sbml=\"http://www.sbml.org/sbml/level3/version1/core\">\n  <rdf:RDF xmlns:rdf=\"http://www.w3.org/1999/02/22-rdf-syntax-ns#\">\n    <rdf:Description rdf:about=\"#M_14glucan_c\">\n      <bqbiol:is xmlns:bqbiol=\"http://biomodels.net/biology-qualifiers/\">\n        <rdf:Bag>\n          <rdf:li rdf:resource=\"http://identifiers.org/bigg.metabolite/14glucan\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/biocyc/META:1-4-alpha-D-Glucan\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/metanetx.chemical/MNXM2905\"/>\n          <rdf:li rdf:resource=\"http://identifiers.org/seed.compound/cpd21754\"/>\n        </rdf:Bag>\n      </bqbiol:is>\n    </rdf:Description>\n  </rdf:RDF>\n</sbml:annotation>")

In [121]:
all_metabolites_list = Vector{String}();
all_metabolites_names = Vector{String}();
for (metabolite_name, metabolite_dict) in metabolites
    metabolite_real_name = metabolite_dict.name;
    push!(all_metabolites_list, metabolite_name);
    push!(all_metabolites_names, metabolite_real_name);
end

In [128]:
all_metabolites_list

1668-element Vector{String}:
 "M_pg120_p"
 "M_galctn__L_c"
 "M_cdec3eACP_c"
 "M_pgp120_p"
 "M_isetac_e"
 "M_4ampm_c"
 "M_dxyl_c"
 "M_phe__L_e"
 "M_ps141_c"
 "M_lys__L_e"
 "M_2pg_c"
 "M_4c2me_c"
 "M_dtdp4addg_c"
 ⋮
 "M_feoxam_p"
 "M_ocdcaACP_c"
 "M_lcts_p"
 "M_so2_p"
 "M_halipa_e"
 "M_acald_c"
 "M_no3_c"
 "M_5apru_c"
 "M_idp_c"
 "M_aspsa_c"
 "M_kdo2lipid4_c"
 "M_2agpg180_p"

In [174]:
reactions["R_DMSOR2pp"].gene_product_association.terms[1]

SBML.GPARef("G_b1872")

In [179]:
function get_genes(gene_assoc)
    reaction_gene_names = Vector{String}();
    if typeof(gene_assoc) == SBML.GPARef
        reaction_gene_id = gene_assoc.gene_product;
        reaction_gene_name = gene_products[reaction_gene_id].name;
        push!(reaction_gene_names, reaction_gene_name);
    elseif typeof(gene_assoc) == SBML.GPAAnd
        reaction_assocs = gene_assoc.terms;
        reaction_gene_names = Vector{String}();
        for reaction_assoc in reaction_assocs
            reaction_assoc_genes = get_genes(reaction_assoc);
            append!(reaction_gene_names, reaction_assoc_genes);
        end
    elseif typeof(gene_assoc) == SBML.GPAOr
        reaction_assocs = gene_assoc.terms;
        reaction_gene_names = Vector{String}();
        for reaction_assoc in reaction_assocs
            reaction_assoc_genes = get_genes(reaction_assoc);
            append!(reaction_gene_names, reaction_assoc_genes);
        end
    else
        @show typeof(gene_assoc)
    end
    return reaction_gene_names
end

get_genes (generic function with 1 method)

In [215]:
default_lbs = zeros(0);
default_ubs = zeros(0);
reactions_genes_list = Vector{Vector{String}}();
for (reaction_name, reaction_dict) in reactions
    reaction_reactants_dict = reaction_dict.reactants;
    reaction_products_dict = reaction_dict.products;
    reaction_genes_assoc = reaction_dict.gene_product_association;
    all_teaction_genes = get_genes(reaction_genes_assoc);
    push!(reactions_genes_list, all_teaction_genes);
    reaction_lb = reaction_dict.lb[1];
    reaction_ub = reaction_dict.ub[1];
    push!(default_lbs, reaction_lb);
    push!(default_ubs, reaction_ub);
end

typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_as

typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_assoc) = Nothing
typeof(gene_as

In [236]:
reactions_genes_list

2388-element Vector{Vector{String}}:
 ["amyA"]
 ["torZ", "torY"]
 []
 ["yjjX"]
 ["phoE", "ompC", "ompN", "ompF"]
 ["nudK"]
 ["arnT"]
 ["fadE"]
 ["frlA"]
 ["tdcC", "sdaC"]
 ["ppc"]
 ["ompF", "phoE", "ompN", "ompC"]
 ["alsE"]
 ⋮
 ["crr", "ptsH", "ptsI", "treB"]
 ["ompC", "ompN", "phoE", "ompF"]
 ["msbA"]
 ["gshB"]
 ["ilvA", "sdaB", "sdaA", "tdcB"]
 []
 []
 ["ugpQ"]
 ["tesA"]
 ["fadJ", "fadB"]
 ["gatB", "ptsH", "ptsI", "gatC", "gatA"]
 []

### Generate L and U

In [208]:
repeat(default_lbs,1, 4)

2388×4 Matrix{Float64}:
       0.0        0.0        0.0        0.0
       0.0        0.0        0.0        0.0
       0.0        0.0        0.0        0.0
       0.0        0.0        0.0        0.0
 -999999.0  -999999.0  -999999.0  -999999.0
       0.0        0.0        0.0        0.0
       0.0        0.0        0.0        0.0
 -999999.0  -999999.0  -999999.0  -999999.0
       0.0        0.0        0.0        0.0
 -999999.0  -999999.0  -999999.0  -999999.0
       0.0        0.0        0.0        0.0
 -999999.0  -999999.0  -999999.0  -999999.0
 -999999.0  -999999.0  -999999.0  -999999.0
       ⋮                          
       0.0        0.0        0.0        0.0
 -999999.0  -999999.0  -999999.0  -999999.0
       0.0        0.0        0.0        0.0
       0.0        0.0        0.0        0.0
       0.0        0.0        0.0        0.0
       0.0        0.0        0.0        0.0
       0.0        0.0        0.0        0.0
       0.0        0.0        0.0        0.0
       0.0       

In [217]:
num_experiments = length(growth_genes_standard_names);
num_reactions = length(reactions_list);
L = repeat(default_lbs,1, num_experiments);
U = repeat(default_ubs,1, num_experiments);

In [237]:
num_experiments

3058

In [234]:
function get_reactions_indices(gene_name)
    apearance_indices = zeros(Int64, 0);
    for j in 1:num_reactions
        gene_list = reactions_genes_list[j];
        if gene_name in gene_list
            push!(apearance_indices, j);
        end
    end
    return apearance_indices
end

get_reactions_indices (generic function with 1 method)

In [235]:
for c in 1:num_experiments
    knocked_out_gene = growth_genes_standard_names[c];
    reaction_indices = get_reactions_indices(knocked_out_gene);
    @show reaction_indices
    for i in reaction_indices
        L[i,c] = 0;
        U[i,c] = 0;
    end
end

reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [46, 211, 238, 263, 273, 417, 661, 702, 1135, 1164, 1207, 1226, 1547, 1592, 1670, 1681, 1718, 1771, 1808, 2106, 2196, 2276, 2279]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [173]
reaction_indices = [1601]
reaction_indices = [262]
reaction_indices = [846]
reaction_indices = Int64[]
reaction_indices = [1805, 2097]
reaction_indices = [1184, 1805, 2097]
reaction_indices = [14, 94, 999, 1484, 1852, 1939, 2273]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [1199]
reaction_indices = [1199]
reaction_indices = Int64[]
reaction_indices = [1084]
reaction_indices = [834, 1402]
reaction_indices = Int64[]
reaction_indices = [494, 210

reaction_indices = [68]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [166, 758, 808, 1634]
reaction_indices = Int64[]
reaction_indices = [571]
reaction_indices = [1029]
reaction_indices = [1031]
reaction_indices = [766]
reaction_indices = [1341]
reaction_indices = [1845]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [1591]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [410, 2113]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int

reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [325, 916, 1572]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [325, 916, 1572]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [325, 916, 1572]
reaction_indices = Int64[]
reaction_indices = [325, 1281]
reaction_indices = [325, 1281]
reaction_indices = [325, 1281]
reaction_indices = [325, 1281]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [1281]
reaction_indices = [1281]
reaction_indices = [1281]
reaction_indices = [1281]
reaction_indices = [1281]
reaction_indices = [1281]
reaction_indices = [1281]
reaction_indices = [1281]
reaction_indices = [1281]
reaction_indices = Int64[]
reaction_indices = [249]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [164

reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [385]
reaction_indices = [385]
reaction_indices = Int64[]
reaction_indices = [100, 628, 1083, 2333]
reaction_indices = [281, 914, 1478, 1576]
reaction_indices = [281, 914, 1478, 1576]
reaction_indices = [100, 628, 1083, 2333]
reaction_indices = [340, 1086]
reaction_indices = [340, 1086]
reaction_indices = [340, 1086]
reaction_indices = [340, 1086]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [2282]
reaction_indices = [2338]
reaction_indices = Int64[]
reaction_indices = [617, 2189]
reaction_indices = [617, 1149, 1223]
reaction_indices = [237, 531]
reaction_indices = [6]
reaction_indices = [579, 1522, 1864]
reaction_indices = [579, 1522, 1864]
reacti

reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [993]
reaction_indices = [993]
reaction_indices = Int64[]
reaction_indices = [171, 311, 336, 429, 446, 468, 485, 513, 535, 544, 576, 715, 935, 1006, 1021, 1095, 1152, 1274, 1277, 1372, 1440, 1490, 1513, 1664, 1690, 1702, 1740, 1764, 1772, 1870, 1910, 1953, 1994, 2016, 2062, 2184, 2246, 2277, 2316, 2324, 2349, 2350]
reaction_indices = [85, 112, 157, 354, 411, 426, 504, 515, 539, 566, 679, 734, 852, 949, 987, 1042, 1134, 1187, 1240, 1262, 1320, 1450, 1695, 1722, 1821, 1823, 1836, 1875, 2004, 2032, 2063, 2090, 2094, 2147, 2323]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [1672]
reaction_indices = [2208]
reaction_indices = Int64[]
reaction_indices = [1183, 1578]
reaction_indice

reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [30, 71, 170, 667, 939, 1043, 1044, 1063, 1178, 1229, 1347, 1435, 1462, 1514, 1724, 1816, 2038, 2098, 2107, 2214, 2385]
reaction_indices = [376, 673, 688, 938, 943, 1688, 1715, 1971, 2064, 2307]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [786]
reaction_indices = [850]
reaction_indices = [225, 1077]
reaction_indices = [841]
reaction_indices = [998]
reaction_indices = [998]
reaction_indices = [904]
reaction_indices = [825]
reaction_indices = [786]
reaction_indices = [786]
reaction_indices = Int64[]
reaction_indices = [1235, 1623]
reaction_indices = [1321, 2059]
reaction_indices = Int64[]
reaction_indices = [1065, 1603]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [62, 549, 

reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [2335]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = [1038, 1151]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64

reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
reaction_indices = Int64[]
r

## API Call

In [78]:
function make_API_call(url)
    try
        response = HTTP.get(url);
        return String(response.body)
    catch e
        return "Error occurred : $e"
    end
end

make_API_call (generic function with 1 method)

In [79]:
response = make_API_call("https://websvc.biocyc.org/getxml?id=ECOLI:G7686")

"<?xml version=\"1.0\" encoding=\"iso-8859-1\"?>\n<ptools-xml ptools-version='25.0' xml:base='http://BioCyc.org/getxml?ECOLI:G7686'><metadata><url>http://BioCyc.org/</url>\n<service_name>getxml</service_name>\n<query>ECOLI:G7686</query>\n<num_results>1</num_results>\n<PGDB orgid='ECOLI' version='25.1'><species datatype='string'>Escherichia coli</species>\n<strain datatype='string'>K-12 substr. MG1655</strain>\n<dblink><dblink-db>NCBI-TAXONOMY-DB</dblink-db>\n<dblink-oid>511145</dblink-oid>\n<dblink-relationship>unification</dblink-relationship>\n<dblink-URL>http://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?mode=Info&amp;id=511145</dblink-URL>\n</dblink>\n</PGDB>\n</metadata>\n<Gene ID='ECOLI:G7686' orgid='ECOLI' frameid='G7686' detail='full'><parent><Gene resource='getxml?ECOLI:BC-4' orgid='ECOLI' frameid='BC-4' class='true'/></parent>\n<parent><Gene resource='getxml?ECOLI:BC-6.1' orgid='ECOLI' frameid='BC-6.1' class='true'/></parent>\n<replicon><Genetic-Element resource='getxml?

In [ ]:
 HTTP.request(
    "GET", 
    "https://websvc.biocyc.org/st-service-get?id=biocyc17-53978-3837907458&format=json"; 
    headers = usr
)

In [ ]:
json_response = JSON.parse(response)

In [ ]:
response_keys = collect(keys(json_response));
json_response[response_keys[3]]

In [ ]:
JSON to table

In [ ]:
xdoc = parse_string(response)

In [ ]:
xdoc = LightXML.parse_string(response)

In [ ]:
function xout(xelement, mode)
    if mode == 1
        println("<",name(xelement), ">")
    else
        println("\t{", name(xelement), "}\t", content(xelement))
    end
end

In [ ]:
xroot = LightXML.root(xdoc)  # an instance of XMLElement
# print its name
println(name(xroot))  # this should print: bookstore

# traverse all its child nodes and print element names
for c in child_nodes(xroot)  # c is an instance of XMLNode
    println(nodetype(c))
    if is_elementnode(c)
        e = XMLElement(c)  # this makes an XMLElement instance
        println(name(e))
    end
end

In [ ]:
function parse_node(c)
    if is_elementnode(c)
        e = XMLElement(c)
        if length(collect(child_elements(e))) > 0
            xout(e, 1)
            for child in child_nodes(e)
                parse_node(child)
            end
        else
            xout(e, 0)
        end
    end
end

In [ ]:
for c in child_nodes(xroot)
    parse_node(c)
end

In [ ]:
headers = (("X-Application","appKey"),("X-Authentication","sessionToken"),
           ("content-type","application/json"))

url="https://postman-echo.com/post"

req = Dict("jsonrpc" => "2.0", "params" => Dict("filet" => Dict()))

response = HTTP.post(url, headers, JSON.json(req))
response_text = String(response.body)
json_obj = JSON.parse()
